<a href="https://colab.research.google.com/github/hubingqian2011/Statistical-machine-learning/blob/main/Kaggle_Google_AI4Code_transformers_AutoModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to Load Kaggle Datasets Directly into Google Colab? https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download AI4Code

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 99% 706M/714M [00:04<00:00, 206MB/s]
100% 714M/714M [00:04<00:00, 166MB/s]


In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 9.3 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=6fee78604a417473e78f2d4ff8e5d4ceed10c4510deef8ccf03265661deb8a1e
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 4.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=5233ed6f64d613fac4c92bbd9f2fe2000620215e8849726493baab21f27f5ba3
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [2]:
import zipfile
with zipfile.ZipFile("AI4Code.zip","r") as z:
  z.extractall() 
  import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120 

# read all json files and add them together
data_dir = Path('/content')

def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )

paths_train = list((data_dir / 'train').glob('*.json'))
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
df.head()

Train NBs: 100%|██████████| 139256/139256 [11:42<00:00, 198.26it/s]


cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
00001756c60be8 1862f0a6      code  # This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
               2a9e43d6      code  import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...
               038b763d      code                                                                       import warnings\nwarnings.filterwarnings('ignore')
               2eefe0ef      code                                                                            matplotlib.rcParams.update({'font.size': 14})
               0beab1cd      code  def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n    print("Train R2:\...

In [ ]:
# example of one file 00001756c60be8
print("The disordered notebook:")
nb_id = "c83c75c65995b1" # c83c75c65995b1
#nb = df.loc[nb_id, :]["cell_type"] == "markdown"
nb = df.loc[nb_id, :]
display(nb)
print()

The disordered notebook:


,cell_type,source
cell_id,,
a0697c01,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...
4e668f62,code,import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom IPython.displa...
a4d6e69a,code,import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom IPython.displa...
59c7069f,code,import numpy as np\nimport matplotlib.pyplot as plt\nimport pandas as pd \nimport seaborn as sns\nfrom IPython.displ...
3508ee47,code,import numpy as np\nimport matplotlib.pyplot as plt\nimport pandas as pd \nimport seaborn as sns\nfrom IPython.displ...
6a55d756,markdown,"**This Data is to analyse the frequency of position in a particular year. This means in that particular year ,which ..."


In [ ]:
from googletrans import Translator
from langdetect import detect

translator = Translator()

# check each id's markdown, translate all of them to English
for i in range(len(notebooks_train)):
  current_file = notebooks_train[i]

  for j in range(len(current_file["cell_type"])):
    if current_file["cell_type"][j] == 'markdown':
      if type(current_file["source"][j]) == str:
        check_language = detect(current_file["source"][j])
        if check_language == 'en':
          break
        else:
          notebooks_train[i]["source"][j] = translator.translate(current_file["source"][j])

trainning_data = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
trainning_data.head()

LangDetectException: ignored

In [ ]:
# example of one file
print("The disordered notebook:")
nb_id = "00001756c60be8"
#nb = df.loc[nb_id, :]["cell_type"] == "markdown"
nb = trainning_data.loc[nb_id, :]
display(nb)
print()

139256

# Data Preprocessing

*   Regular Expression/Normalization — lowercase the words, remove punctuation and remove numbers 
*   Tokenization — a process of splitting the text into smaller pieces called tokens
*   Stop Words Removal — a set of commonly used words in any language
*   Lemmatization — a process of grouping together the inflected forms of a word so they can be analyzed as a single item
*   Stemming — a process of grouping together the inflected forms of a word so they can be analyzed as a single item

In [3]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list
df_orders.head()

id
00001756c60be8    [1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b763d, 77e56113, 2eefe0ef, 1ae087ab, 0beab1cd, 8ffe0b25, 9a78ab76, 0d136...
00015c83e2717b    [2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195f8b, 25699d02, 72b3201a, f2c75...
0001bdd4021779    [3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310c80, 073e27e5, 015d52a4, ad7679ef, 7fde4f04, 07c52510, 0a1a7a39, 0bcd3...
0001daf4c2c76d    [97266564, a898e555, 86605076, 76cc2642, ef279279, df6c939f, 2476da96, 00f87d0a, ae93e8e6, 58aadb1d, d20b0094, 986fd...
0002115f48f982                                 [9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a]
Name: cell_order, dtype: object

In [4]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)
df_orders_.head()

,cell_order,cell_id
id,,
00001756c60be8,"[1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b763d, 77e56113, 2eefe0ef, 1ae087ab, 0beab1cd, 8ffe0b25, 9a78ab76, 0d136...","[1862f0a6, 2a9e43d6, 038b763d, 2eefe0ef, 0beab1cd, 9a78ab76, ebe125d5, d9dced8b, 86497fe1, e2c8e725, ff7c44ed, 0e7c9..."
00015c83e2717b,"[2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195f8b, 25699d02, 72b3201a, f2c75...","[c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195f8b, 25699d02, de148b56, 9901472c, 10377ef8, 1f462e2f, fceeb3e6, 2af2a..."
0001bdd4021779,"[3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310c80, 073e27e5, 015d52a4, ad7679ef, 7fde4f04, 07c52510, 0a1a7a39, 0bcd3...","[3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310c80, 073e27e5, 015d52a4, ad7679ef, 07c52510, 0a1a7a39, 0bcd3fef, 7fde4..."
0001daf4c2c76d,"[97266564, a898e555, 86605076, 76cc2642, ef279279, df6c939f, 2476da96, 00f87d0a, ae93e8e6, 58aadb1d, d20b0094, 986fd...","[86605076, df6c939f, 00f87d0a, 986fd4f1, 9b761026, 6f271c86, 97c3f99b, cfa510c5, 374a5179, df5f7c1f, 27060ab9, 4c739..."
0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a]","[18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a, 9ec225f0]"


In [5]:
ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}
df_ranks = (
    pd.DataFrame
    .from_dict(ranks, orient='index')
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)
df_ranks.head()

rank
id             cell_id      
00001756c60be8 1862f0a6    0
               2a9e43d6    2
               038b763d    4
               2eefe0ef    6
               0beab1cd    8

In [6]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
df = df.reset_index().merge(df_ranks, on=["id", "cell_id"]).merge(df_ancestors, on=["id"])
df["pct_rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")
df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/pyt...,0,945aea18,NaN,0.0
1,00001756c60be8,2a9e43d6,code,"import numpy as np\nimport pandas as pd\nimport random\n\nfrom sklearn.model_selection import train_test_split, cros...",2,945aea18,NaN,0.034483
2,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('ignore'),4,945aea18,NaN,0.068966
3,00001756c60be8,2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),6,945aea18,NaN,0.103448
4,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):\n print(""Train R2:\...",8,945aea18,NaN,0.137931
...,...,...,...,...,...,...,...,...
6370641,fffe1d764579d5,0d770d6b,markdown,## REMOVING THE OUTLIERS,43,3c40bfa6,NaN,0.597222
6370642,fffe1d764579d5,d45ddc62,markdown,### DIMENSIONALITY CURSE,33,3c40bfa6,NaN,0.458333
6370643,fffe1d764579d5,1a63248d,markdown,# BANGALORE HOUSE PRICE PREDICTION,0,3c40bfa6,NaN,0.0
6370644,fffe1d764579d5,a8ffc8b4,markdown,* We have achieved 75.2% accuracy in predicting the prices of the homes in Banglore using Decision Tree Regressor,69,3c40bfa6,NaN,0.958333


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)
train_ind, val_ind = next(splitter.split(df, groups=df["ancestor_id"]))
train_df = df.loc[train_ind].reset_index(drop=True)
val_df = df.loc[val_ind].reset_index(drop=True)

# Base markdown dataframes
train_df_mark = train_df[train_df["cell_type"] == "markdown"].reset_index(drop=True)
val_df_mark = val_df[val_df["cell_type"] == "markdown"].reset_index(drop=True)

In [ ]:
nb_id = "00290ddf866418"
hu = df_train.loc[nb_id, :]['source'].astype(str)[1]
hu

"import warnings\nimport random\n\nSEED=44\nrandom.seed(SEED)\nnp.random.seed(SEED)\npd.set_option('display.width', None)\npd.set_option('display.max_column', None)\n\nwarnings.filterwarnings('ignore')"

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Training set
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str))

In [ ]:
tfidf.fit_transform([hu])

<1x14 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [ ]:
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()
        #return document

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)
        return preprocessed_text

    
def preprocess_df(df):
    """
    This function is for processing sorce of notebook
    returns preprocessed dataframe
    """
    return [preprocess_text(message) for message in df.source]

df.source = df.source.apply(preprocess_text)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)
train_ind, val_ind = next(splitter.split(df, groups=df["ancestor_id"]))
train_df = df.loc[train_ind].reset_index(drop=True)
val_df = df.loc[val_ind].reset_index(drop=True)

# Base markdown dataframes
train_df_mark = train_df[train_df["cell_type"] == "markdown"].reset_index(drop=True)
val_df_mark = val_df[val_df["cell_type"] == "markdown"].reset_index(drop=True)
train_df_mark.to_csv("train_mark.csv", index=False)
val_df_mark.to_csv("val_mark.csv", index=False)
val_df.to_csv("val.csv", index=False)
train_df.to_csv("train.csv", index=False)

In [ ]:
train_df_mark.head()

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,00001756c60be8,21616367,markdown,*Деление признаков на числовые и текстовые*,17,945aea18,NaN,0.293103
1,00001756c60be8,fcb6792d,markdown,**Сортировка признаков по важности**,53,945aea18,NaN,0.913793
2,00001756c60be8,63c26fa2,markdown,"Выбросы наблюдаются в: HouseYear, KitchenSquare.\n\nПризнаки с аномально высоким значением, которые нужно будет огра...",30,945aea18,NaN,0.517241
3,00001756c60be8,4bb2e30a,markdown,**Поиск признаков с выбросами**,28,945aea18,NaN,0.482759
4,00001756c60be8,a6357f7e,markdown,*Выводим сколько строк в тесте и на трейне*,21,945aea18,NaN,0.362069
